In [106]:
import tensorflow as tf
from tensorflow import keras


In [109]:
# Q1
model = tf.keras.models.load_model('bees-wasps.h5', compile=False)
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model =converter.convert()

with open('convertmodel.tflite','wb') as f:
    f.write(tflite_model)
tflite_model.__sizeof__() / 1024000

43.8143681640625

In [111]:
# Q2
lite_model = tf.lite.Interpreter('convertmodel.tflite')
lite_model.allocate_tensors()
# lite_model.get_output_details()

input_indx = lite_model.get_input_details()[0]['index']
output_index = lite_model.get_output_details()[0]['index']


In [115]:
lite_model.get_input_details()

[{'name': 'conv2d_input',
  'index': 0,
  'shape': array([  1, 150, 150,   3]),
  'shape_signature': array([ -1, 150, 150,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [125]:
# Q3
from io import BytesIO
from urllib import request
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
img = download_image('https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg')
prep_img = prepare_image(img,[150,150])

def preprocess(x):
    x /=127.5
    x -=1.
    return x


num_img = np.array(prep_img,dtype='float32')

preprocess(num_img)

train_gen = ImageDataGenerator()
pre_img = train_gen.apply_transform(num_img,transform_parameters={
'theta':50,
'tx':0.1,
'ty':0.1,
'zx':0.1,
'zy':0.1,
'flip_horizontal':True 
})
pre_img = np.array([pre_img])
pre_img[0][0][0][0]

0.35420817

In [130]:
# Q4

lite_model.set_tensor(input_indx,pre_img)
lite_model.invoke()
prep = lite_model.get_tensor(output_index)
prep[0][0]

0.48443806

In [143]:
%%bash
# Q5
docker images

REPOSITORY                      TAG       IMAGE ID       CREATED      SIZE
agrigorev/zoomcamp-bees-wasps   v2        b9f6c13de368   8 days ago   662MB


In [144]:
# Q6
0.5164256

<Response [404]>